# Creators
* Morgan Dally - 1313361
* Reece Breebaart - 1314828

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [ ]:
def print_predicted_stats(predicted, y_test):
    '''
    This is just a dumb version of classification report.
    '''
    hit = 0
    miss = 0
    for actual, prediction in zip(y_test, predicted):
        if actual == prediction:
            hit += 1
            continue
        miss += 1
    percent = 100 - ((miss/hit) * 100)
    print('missed: %d\nhit: %d\n%d%s' % (miss, hit, percent, '% accuracy'))

In [ ]:
input_csv_loc = '../input/wisconsin_breast_cancer.csv'
bccf = pd.read_csv(input_csv_loc)

# get rid of NaN entries
bccf_clean = bccf.dropna()
bccf_clean.head()

In [ ]:
import seaborn as sns
sns.pairplot(data=bccf_clean, hue='thickness', palette='Set2')

In [ ]:
from sklearn.model_selection import train_test_split

# select all entries, then get from col1 and select up until the end, exlcuding it
x_dat = bccf_clean.iloc[:, 1:-1]
# select all entries, then get col 10
y_dat = bccf_clean.iloc[:, 10]

x_train, x_test, y_train, y_test = train_test_split(x_dat, y_dat, test_size=0.2, random_state=342)

print('x_train: %s\ny_train: %s\nx_test: %s\ny_test: %s' % (
    x_train.shape, y_train.shape,
    x_test.shape, y_test.shape))

In [ ]:
from sklearn.svm import SVC
model = SVC()
model.fit(x_train, y_train)

In [ ]:
predicted = model.predict(x_test)
print_predicted_stats(predicted, y_test)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(y_test, predicted))

In [ ]:
print(classification_report(y_test, predicted))

In [ ]:
x_test.head()

# xsub train
## Our xsub col choices:
* nuclei
* size or shape (these two will be related features so try either or)
* adhesion
* single

## Notes / Discussion:
* Class 0 looks to be easier to distinguish from class 1
* We therefore want to select in a way that makes class 1 features more prominent
* We experimented and eyeballed the accuaracy
* tried to determine which classes were detremental to perfomance
* it seemed that selecting columns with a dominant class provided the largest accuracy
* in the end we were able to make a subset that contradicted any results we found trying to find any patterns
* we were not able to determine good or bad individual columns
* we ended up with [5, 2, 3, 4] as they improved accuracy

#### max vals
*  1: 139
*  2: 373 <- 5
*  3: 346
*  4: 393 <- 3
*  5: 376 <- 4 (seems to influence results badly) // contradicted as we made a good subset with it improving performance
*  6: 402 <- 2
*  7: 161
*  8: 432 <- 1

In [ ]:
bccf_clean.iloc[:, 8].value_counts()

In [ ]:
"""
cols:
    thickness: 0,
    size: 1,
    shape: 2,
    adhesion: 3,
    single: 4,
    nuclei: 5,
    chromatin: 6,
    nucleoli: 7,
    mitosis: 8
my choice: [nuclei, shape, adhesion, single]
"""
# selecting rows to increase accuracy - eyeballed 96% accuracy
cols = [5, 2, 3, 4]
# experimenting getting bad accuracy, came out with 92%
# cols = [5, 4, 8, 1]
xsub_train = x_train.iloc[:, cols]
xsub_test = x_test.iloc[:, cols]
xsub_train.head()

In [ ]:
second_model = SVC()
second_model.fit(xsub_train, y_train)

In [ ]:
sub_predicted = second_model.predict(xsub_test)
print_predicted_stats(sub_predicted, y_test)

In [ ]:
print(confusion_matrix(y_test, sub_predicted))

In [ ]:
print(classification_report(y_test, sub_predicted))

# kNN Tests

In [ ]:
import sklearn.neighbors
#when k value is 1
knn_1_model = sklearn.neighbors.KNeighborsRegressor(n_neighbors=1)
knn_1_model.fit(x_train, y_train)

In [ ]:
knn_1_predicted = knn_1_model.predict(x_test)
print_predicted_stats(knn_1_predicted, y_test)

In [ ]:
#when k value is 1
knn_5_model = sklearn.neighbors.KNeighborsClassifier(n_neighbors=5)
knn_5_model.fit(x_train, y_train)

In [ ]:
knn_5_predicted = knn_5_model.predict(x_test)
print_predicted_stats(knn_5_predicted, y_test)

In [ ]:
y_test.head()

In [ ]:
print(classification_report(y_test, knn_1_predicted))

In [ ]:
print(classification_report(y_test, knn_5_predicted))

# Discussion
From the above tests, our results were:
* x_train:
  * 100% accuracy classifying class 0
  * 85% accuracy classifying class 1
* xsub_train:
  * 100% accuracy classifying class 0
  * 92% accuracy classifying class 1
* Nearest neighbor model, k = 1:
  * 97% accuracy classifying class 0
  * 95% accuracy classifying class 1
* Nearest neighbor model, k = 5:
  * 99% accuracy classifying class 0
  * 98% accuracy classifying class 1
  
## All cols and subsetting

Modifying the test and train set to only use a subset of columns saw varying results. In general it seems that with this dataset, subsetting the data can improve accuracy. However it was possible to also degrade the performance of our model. This can be seen with the two subsets 5 code blocks up (one is commented out). In this case subsetting was able to improve the accuracy of classifying class 1 from 85% to 92%, a significant difference.

The largest accuracy difference was actually found by modifying the random_state value when initially splitting into the train and test sets. For example at one point the initial split saw 98% accuracy. After reloading the page the accuracy then dropped to around 95%. To compensate for this the random state used for splitting data has been pinned. In both cases subsetting the data could improve the accuracy of classifying the classes, moreso classifying class 1.

For more conclusive results we would need to test with a larger dataset. However it can be said that subsetting improved our performance with a 'good' subset.

## kNN Results
* k = 1:
  * 97% accuracy classifying class 0
  * 95% accuracy classifying class 1
* k = 5:
  * 99% accuracy classifying class 0
  * 98% accuracy classifying class 1

Using the neighbor classifier model improved the performance over the original SVC model used. When k = 1, it could be seen that the accuracy improve for classifying class 1 (92% -> 95%), however the accuracy for finding class 0 dropped (100% -> 97%). When setting k = 5 however or accuracy improved greatly. Class 0 was now classified 99% of the time and class 1 was classified 98% of the time.

In conclusion it looks like a nearest neighbor model with k = 5 is the best approach for the provided dataset.
